In [10]:
import pandas as pd
import numpy as np
import nltk
import matplotlib
import matplotlib.pyplot as plt
from pandas import DataFrame
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import math
import random
from sklearn import datasets
from sklearn import neighbors
from sklearn.preprocessing import Imputer

In [11]:
#import pandas.io.data
data = pd.DataFrame()

In [12]:
def dataRead():
    df = pd.read_csv('new_train.csv',skipinitialspace=True)
    #print df.head()

    null_rows = df[df.isnull().any(axis=1)]# All the rows that have a null value

    null_cols = df.columns[df.isnull().any()].tolist()# Alll the columns that have a null value
    
    

    print ('Null_rows')
    print (null_rows)

    print ('Null_columns')
    print (null_cols)
    #print type(df['site'][3])

    return df

In [13]:
def impute(mat, learner, n_iter=3):
    mat = np.array(mat)
    mat_isnan = np.isnan(mat)        
    w = np.where(np.isnan(mat))
    ximp = mat.copy()
    for i in range(0, len(w[0])):
        n = w[0][i] # row where the nan is
        p = w[1][i] # column where the nan is
        col_isnan = mat_isnan[n, :] # empty columns in row n
        train = np.delete(mat, n, axis = 0) # remove row n to obtain a training set
        train_nonan = train[~np.apply_along_axis(np.any, 1, np.isnan(train)), :] # remove rows where there is a nan in the training set
        target = train_nonan[:, p] # vector to be predicted
        feature = train_nonan[:, ~col_isnan] # matrix of predictors
        learn = learner.fit(feature, target) # learner
        ximp[n, p] = learn.predict(mat[n, ~col_isnan].reshape(1, -1)) # predict and replace
    for iter in range(0, n_iter):
        for i in random.sample(range(0, len(w[0])), len(w[0])):
            n = w[0][i] # row where the nan is
            p = w[1][i] # column where the nan is
            train = np.delete(ximp, n, axis = 0) # remove row n to obtain a training set
            target = train[:, p] # vector to be predicted
            feature = np.delete(train, p, axis=1) # matrix of predictors
            learn = learner.fit(feature, target) # learner
            ximp[n, p] = learn.predict(np.delete(ximp[n,:], p).reshape(1, -1)) # predict and replace
    
    return ximp

In [14]:
def dataAnalysis():
    df = dataRead()
    #df['VEHICLE_TYPE'] = map(lambda x: x.upper(), df['VEHICLE_TYPE'])
    
    '''we set a working frame'''
    wf = df
    print (wf.shape)
    
    '''we start with identifying the columns in which data is missing'''
    null_cols = df.columns[df.isnull().any()].tolist()# Alll the columns that have a null value
    all_cols = df.columns
    
    print ("Null_Columns:")
    print (null_cols)
    
    print ("All Columns:")
    print (all_cols)
    
    '''Next we need to calculate the perccentage of dataset that has missing data points'''
    
    total_cells = df.shape[0]*df.shape[1]
    n_empty_cells = df.isnull().values.ravel().sum()   #total number of empty cells
    
    n_rows = df.shape[0]
    n_empty_rows = df.shape[0] - wf.dropna().shape[0]
    
    percentage_null_cells = float(n_empty_cells)/total_cells
    percentage_null_rows = float(n_empty_rows)/n_rows
    
    #print n_empty_cells
    #print n_empty_rows
    print ('% of empty rows: ',percentage_null_rows)
    print ('% of empty cells: ',percentage_null_cells)    
    print (wf.describe())
    
    wf['Garden'].fillna(method='ffill',inplace=True)
    wf['King_visit'].fillna(method='ffill',inplace=True)
    
    n_rows_wf = wf.shape[0]
    n_empty_rows_wf = wf.shape[0] - wf.dropna().shape[0]
    
    percentage_null_rows_wf = float(n_empty_rows_wf)/n_rows
    
    #print n_empty_cells
    #print n_empty_rows
    print ('% of empty rows of wf: ',percentage_null_rows_wf)
    
    wf.drop('Garden',axis=1)
    wf.drop('King_visit',axis=1)
    
    '''for column in wf.columns:
        if wf[column].dtype != np.float64 and wf[column].dtype != np.int64:
            print column
            print wf[column].unique().tolist()
         '''
    
    #===============MAPPINGS=================
    mapping_FLand = {'small':0, 'huge':1}
    wf.replace({'Farm_land':mapping_FLand}, inplace = True)
    
    mapping_Loc = {"King's Landing":0, 'The Mountains':1, 'Cursed Land':2, "Servant's Premises":3}
    wf.replace({'Location':mapping_Loc}, inplace = True)
    
    mapping_Builders = {'Bob':0, 'Bright_Brothers':1, 'Masters_of_Stones':2, 'The_Greens':3, 'The_Kings':4, 'The_Lannisters':5, 'The_Ollivers':6, 'The_Overlords':7, 'The_Starks':8, 'Wood_Priests':9, 'Not_Known':10}
    wf.replace({'Builder':mapping_Builders}, inplace = True)
    
    
    wf_KNN_impute = impute(mat=np.array(wf.drop('House_ID',axis=1)), learner=neighbors.KNeighborsRegressor(n_neighbors=3), n_iter=10)
    DataFrame(wf_KNN_impute).to_csv('KNN_imputed.csv')
    
    imp = Imputer(strategy = 'mean',axis=1)
    wf_mode_imput = imp.fit_transform(np.array(wf.drop('House_ID',axis=1)))
    DataFrame(wf_mode_imput).to_csv('ModeImputed.csv') 
    
    wf_lin_interpol = wf.interpolate()
    wf_lin_interpol.to_csv('LinearInterpolated.csv')
    valid_vars = ['Built_Date','Priced_Date','Garden','Dock_Distance','Capital_Distance',
    'Royal_Distance','Guarding_Distance','River_distance','Renovation','Dining_Rooms','Bedroom',
    'Bathroom','King_visit','Curse','Blessing','Knight_Distance','Golden_Grains']
    
    '''raw_fig = sns.pairplot(df)
    raw_fig.save('RawDataPlot.png')
    plt.show()'''
    
    '''print 'Modified Datframe with linear interpolation'
    
    print wf_lin_interpol.describe()'''
    
    '''for i in null_cols :
        plt.plot(wf[i])
        plt.show()'''
    
    '''interpol_lin_fig = sns.pairplot(wf_lin_interpol,hue='Builder',vars=valid_vars)
    interpol_lin_fig.savefig('InterpolatedLinPlot.png')
    plt.show()'''
    
    #ax = sns.heatmap(wf_lin_interpol[['Dock_Distance','Capital_Distance','Royal_Distance','Golden_Grains']].copy(), annot=True, fmt = "d")
    #heatmap_fig.savefig('Heatmap.png')
    #plt.show()
    
    #Next we try to club-up certain columns like Built-date-time and sold-date-time as index
    
    #Next we try to interpolate using different techniques and then find correlations between different features and try yo figure out what kind of interpolation fits which feature
    
    '''wf_quad_interpol = wf.interpolate(method='quadratic').set_index('House_ID')
    wf_quad_interpol.to_csv('QuadInterpolated.csv')
    
    wf_slinear_interpol = wf.interpolate(method = 'slinear').set_index('House_ID')
    wf_slinear_interpol.to_csv('slinearInterpolated.csv')
    
    wf_barycentric_interpol = wf.interpolate(method = 'barycentric').set_index('House_ID')
    wf_barycentric_interpol.to_csv('barycentricInterpolated.csv')
    
    wf_pPoly_interpol = wf.interpolate(method = 'piecewise_polynomial').set_index('House_ID')
    wf_pPoly_interpol.to_csv('PiecewisePolynomialInterpolated.csv')
    
    wf_pchip_interpol = wf.interpolate(method = 'pchip').set_index('House_ID')
    wf_pchip_interpol.to_csv('pchipInterpolated.csv')
    
    wf_spline_interpol = wf.interpolate(method = 'spline', order = 3).set_index('House_ID')
    wf_spline_interpol.to_csv('splineInterpolated.csv')'''
    
    
    '''interpol_quad_fig = sns.pairplot(wf_quad_interpol,hue='Builder',vars = ['Dock_Distance','Capital_Distance','Royal_Distance','Golden_Grains'])
    interpol_quad_fig.savefig('InterpolatedQuadPlot.png')
    plt.show()'''
    
    
    
    
    
    
    
    
    
    
    #Next we try to identify if there is any relationship between missing data values and any other feature
    #We do this by investigating the values of non-missing features and features with missing values
    
    '''
    tot_vehicle_types = df.VEHICLE_TYPE.unique().tolist()
    
    #Vehicle-Type and Wait-Time
    index = df['WAIT_TIME'].index[df['WAIT_TIME'].apply(np.isnan)]
    df_index = df.index.values.tolist()
    wait_time_idx = [df_index.index(i) for i in index]
    print set([df['VEHICLE_TYPE'][i] for i in wait_time_idx])
    print tot_vehicle_types'''
    
    
    #vehicle-type and total-luggage weight
    #---COPY CODE FROM ABOVE--
    
    #Find correlation b/w wait-time and fare and total-luggage-weight and fare
    #---WRITE CODE--
    
    
    #This code is used to see correlations b/w the columns in which values were missing and fare, at the points where the values were present
    '''temp_f = wf.dropna()
    sns.pairplot(temp_f, hue='VEHICLE_TYPE', x_vars = 'FARE',y_vars = ['WAIT_TIME','TOTAL_LUGGAGE_WEIGHT'], kind = 'reg')
    plt.show()
    #sns_plot = sns.pairplot(temp_f, hue='VEHICLE_TYPE', x_vars = 'FARE',y_vars = ['WAIT_TIME','TOTAL_LUGGAGE_WEIGHT'], kind = 'reg')
    #sns_plot.savefig('output.png')
    sns.pairplot(temp_f,vars = ['WAIT_TIME','TOTAL_LUGGAGE_WEIGHT','FARE'], kind = 'reg')
    plt.show()
    sns.pairplot(temp_f,hue = 'VEHICLE_TYPE',vars = ['WAIT_TIME','TOTAL_LUGGAGE_WEIGHT','FARE'], kind = 'reg')
    plt.show()'''
    
    #This was used only for checking. Hence, not needed anymore
    '''
    print 'Now We check if wait-time and total-luggage-weight go null simultaneously'
    for index, row in df.iterrows():
        if math.isnan(row['WAIT_TIME']) and not math.isnan(row['TOTAL_LUGGAGE_WEIGHT']):
            print row["WAIT_TIME"], row["TOTAL_LUGGAGE_WEIGHT"]
            '''
        
    
    #Now we remove the wait-time and total-luggage-weight and check for percentage of missing data points
    '''wf2 = df.drop(['WAIT_TIME','TOTAL_LUGGAGE_WEIGHT'],axis=1)
    print 'New workframe after deleting waait-time and total-luggage-weight'
    print wf2.shape
    
    wf2_n_rows = wf2.shape[0]
    wf2_n_empty_rows = wf2.shape[0] - wf2.dropna().shape[0]
    wf2_missing_row_ratio = float(wf2_n_empty_rows)/wf2_n_rows
    
    print 'New percentage of empty rows : ', wf2_missing_row_ratio'''
    
     
    #Now we pairplot the remaining data and visualize the relations
    
    '''FDA_plot1 = sns.pairplot(wf2.dropna(),hue='VEHICLE_TYPE',vars=['ID','STARTING_LATITUDE','STARTING_LONGITUDE','DESTINATION_LATITUDE', 'DESTINATION_LONGITUDE','FARE'])#Do it in 2 groups
    FDA_plot1.savefig('FDAplot1.png')
    plt.show()
    
    
    FDA_plot2 = sns.pairplot(wf2.dropna(),hue='VEHICLE_TYPE',vars=['TRAFFIC_STUCK_TIME','DISTANCE', 'FARE'])
    FDA_plot2.savefig('FDAplot2.png')
    plt.show()'''
    
    
    
    return wf
    

In [15]:
#Processing by filling the missing datapoints
def dataProcess():
    tf = dataAnalysis() #temporary frame
    Ff = tf.dropna()
    
    fill_source_latitude = pd.DataFrame()
    fill_source_longitude = pd.DataFrame()
    fill_dest_latitude = pd.DataFrame()
    fill_dest_longitude = pd.DataFrame()
    
    for index,row in tf.iterrow():
        if math.isnan(row['STARTING_LATITUDE']):
            temp = pd.DataFrame([row['STARTING_LONGITUDE'],row['DESTINATION_LATITUDE'],row['DESTINATION_LONGITUDE'],row['DESTINATION_LATITUDE'],row['DISTANCE']], columns = ['STARTING_LONGITUDE', 'DESTINATION_LATITUDE', 'DESTINATION_LONGITUDE', 'DISTANCE'] )
            fill_source_latitude.append(temp)
        
        elif math.isnan(row['STARTING_LONGITUDE']):
            temp = pd.DataFrame([row['STARTING_LONGITUDE'],row['DESTINATION_LATITUDE'],row['DESTINATION_LONGITUDE'],row['DESTINATION_LATITUDE'],row['DISTANCE']], columns = ['STARTING_LONGITUDE', 'DESTINATION_LATITUDE', 'DESTINATION_LONGITUDE', 'DISTANCE'])
    
    #tf['TIMESTAMP'] = np.array(tf['TIMESTAMP'].astype('np.datetime64[s]').tolist())
    tf['TIMESTAMP'] = map(lambda x: pd.to_datetime(x), tf['TIMESTAMP'])
    #tf['TIMESTAMP'] = np.array(tf['TIMESTAMP'])
    
    tf['YEAR'] = map(lambda dt: dt.date().year, tf['TIMESTAMP'])
    tf['MONTH'] = map(lambda dt: dt.date().month, tf['TIMESTAMP'])
    tf['DAY'] = map(lambda dt: dt.date().day, tf['TIMESTAMP'])
    tf['HOUR'] = map(lambda dt: dt.time().hour, tf['TIMESTAMP'])
    tf['MINUTE'] = map(lambda dt: dt.time().minute, tf['TIMESTAMP'])
    tf['SECOND'] = map(lambda dt: dt.time().second, tf['TIMESTAMP'])
    
    del tf['TIMESTAMP']
    
    print (tf['YEAR'])
    
    mapping = {'BUS':0, 'TAXI AC':1, 'TAXI NON AC':2, 'METRO':3, 'AC BUS':4, 'MINI BUS':5, 'AUTO RICKSHAW':6}
    tf.replace({'VEHICLE_TYPE':mapping}, inplace = True)
    
    '''FDA_plot3 = sns.pairplot(tf,kind='reg')
    FDA_plot3.savefig('FDA_plot3.png')
    plt.show()'''
    
    
    
    print (tf2[1])
    print (type(tf2))
    
    return tf2


In [ ]:
def try1():
    df = dataProcess()
    
    print (type(df[1][0]))
    #plt.plot(df[1],df[9])
    #plt.show()


In [ ]:
dataAnalysis()

Null_rows
       House_ID    Built_Date   Priced_Date  Garden  Dock_Distance  \
0      6e32cec0   42116021241   70916090111     0.0     138.600639   
3      6e32cfa9   81816061223   12016081045     0.0      82.283966   
4      6e32cfb7   91116031153   52016111649     0.0      34.524141   
5      6e32cfd5   61316011325   61916110413     NaN      38.877415   
7      6e32d0f3  112216000000   11316111132     0.0      83.914728   
8      6e32d166   10716020334   10216071519     0.0      64.288283   
10     6e32d244   32216040245  100816000000     0.0      74.890453   
11     6e32d266   32116030745   91116111314     0.0       1.624838   
13     6e32d28a   21516060848   11116080913     NaN            NaN   
14     6e32d2e0   91916041728  101916000000     0.0      86.072564   
15     6e32d315   30716020221   51816071132     0.0      51.245386   
16     6e32d328  120616000000  101016000000     0.0      34.410733   
17     6e32d349  110316000000   80216091328     NaN            NaN   
20     6e3